## 1) Fetch weather data - API

In [5]:
import requests 
import pandas as pd

def fetch_weather_from_api(url: str, params: dict) -> pd.DataFrame:
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    hourly = data["hourly"]
    df = pd.DataFrame(hourly)
    df['timestamp'] = pd.to_datetime(df['time'])
    df.set_index('timestamp', inplace=True)
    df.drop(columns=['time'], inplace=True)
    return df

url = "https://api.open-meteo.com/v1/forecast"
params = {
            "latitude": 52.22,
            "longitude": 21.0122,
            "hourly": ["temperature_2m", "relative_humidity_2m", "pressure_msl"],
            "start_date": "2025-11-01",
            "end_date": "2025-11-10"
}

df = fetch_weather_from_api(url, params)
df.to_csv("../data/weather.csv")

## 2) Calculate averages/extremes and detect anomalies

In [ ]:
daily_stats = df.resample('D').agg({
    'temperature_2m': ['mean', 'min', 'max'],
    'relative_humidity_2m': ['mean', 'min', 'max'],
    'pressure_msl': ['mean', 'min', 'max']
})
daily_stats.head()

temperature_2m            relative_humidity_2m          \
                     mean  min   max                 mean min max   
timestamp                                                           
2025-11-01       7.979167  3.4  12.0            81.000000  64  93   
2025-11-02      11.737500  9.6  14.2            86.333333  79  98   
2025-11-03      10.491667  9.6  11.3            91.625000  80  97   
2025-11-04       9.362500  6.9  12.0            80.833333  66  93   
2025-11-05       7.229167  4.0  11.2            79.875000  65  89   

           pressure_msl                  
                   mean     min     max  
timestamp                                
2025-11-01  1019.154167  1015.5  1021.7  
2025-11-02  1012.808333  1011.3  1015.2  
2025-11-03  1014.545833  1010.1  1021.7  
2025-11-04  1024.941667  1022.1  1027.9  
2025-11-05  1025.854167  1023.8  1028.1